# Computational Structural Design II - Materialisation

### Learning Goal: 
- 


### Content:
- 00. Load Thrustdiagram
- 01. Triangulation
- 02. Remesh
- 03. Dual
- 04. Boundary Dual Faces
- 05. Offsets
- 06. Blocks
- 07. Flat Top

### Google Colab: 



---


In [6]:
# % pip install compas_notebook

---
# 0. Load Thrustdiagram

In [7]:
import os
import compas
from google.colab import drive
from google.colab import files

from compas.datastructures  import Mesh
from compas_notebook.app import App


# 1. load thrust json from rv2
thrust_from_rv2 = "https://raw.githubusercontent.com/BlockResearchGroup/CSD2_2022/main/3_Materialization/Tutorial7/scripts/data/square_base_thrust.json"
thrustdiagram_from_rv2 = compas.json_load(thrust_from_rv2)


# 2. extract thrustdiagram data
thrust_data = thrustdiagram_from_rv2['data']['thrust']


# 3. recreate thrustdiagram as a mesh from thrustdiagram data
thrust_mesh = Mesh.from_data(thrust_data)


# 4. export mesh data to a new file in google drive

filename = '00_thrust_mesh.json'

# saving to google drive
drive.mount('/content/drive')
dirname = '/content/drive/My Drive/Colab Notebooks'
googledrive_path = os.path.join(dirname, filename)
compas.json_dump(thrust_mesh, googledrive_path, pretty=True)

# # saving to local
# compas.json_dump(thrust_mesh, filename, pretty=True)
# files.download(filename)


# 5. visualise the mesh
viewer = App()
viewer.add(thrust_mesh)
viewer.show()

ModuleNotFoundError: No module named 'google'

---
# 1. Triangulation

---
# 2. Remesh

---
# 3. Dual

--- 
# 4. Boundary Dual Faces

---
# 5. Offsets 

---
# 6. Blocks

---
# 7. Flat Tops